In [8]:
import numpy as np
import csv
import pandas
import os



In [9]:
'''
read in NTUH AP list and cut the final three characters
e.g '00:be:75:db:92:40 -> '00:be:75:db:92'
'''
BSSID_list = []
with open("./BSSID.txt",'r') as f:
    lines = f.readlines()
    for line in lines:
        BSSID_list.append(line[:-4])
print(BSSID_list)
        



['00:be:75:db:92', '00:be:75:09:d7', '00:be:75:d5:21', '00:be:75:c9:6e', '00:be:75:d0:e2', '00:be:75:dd:6d', '00:be:75:09:dd', '00:be:75:dd:72', '00:be:75:d0:d8', '00:be:75:d5:25', '00:be:75:d5:24', '6c:b2:ae:2d:40', '00:be:75:d5:2c', '00:be:75:dd:74', '00:be:75:d5:33', '00:be:75:dd:75', '00:be:75:d8:79', '00:be:75:d8:74', '00:be:75:d5:1c', '00:be:75:d8:7c', '00:be:75:db:a5', '00:be:75:dd:67', '00:be:75:d0:dc', '00:be:75:d0:e0', '00:be:75:d8:67', '00:be:75:db:95', '00:be:75:d0:d1', '00:be:75:d5:1d', '00:be:75:d5:31', '00:be:75:d0:ce', '00:be:75:d5:2e', '00:be:75:d5:30', '00:be:75:d5:2c', '00:be:75:09:dc', '00:be:75:d5:2c', '00:be:75:d5:1c', '00:be:75:d5:33', '00:be:75:d0:e4', '00:be:75:d8:7d', '00:be:75:d5:25', '00:be:75:db:a4', '00:be:75:db:a7', '00:be:75:d5:32', '00:be:75:d8:7f', '00:be:75:d0:e1', '00:be:75:db:9b', '00:be:75:d5:28', '00:be:75:09:da', '00:be:75:db:a0', '00:be:75:db:a4', '00:be:75:dd:71', '00:be:75:d8:74', '00:be:75:d0:d6', '00:be:75:c9:75', '6c:b2:ae:2d:40', '00:be:75

In [10]:
'''
preprocess raw dataset into fingerprint.txt
args: path list
output:Fingerprint.txt
'''
#for preprocessing raw training datasets
#path_list = ["./Data/Train2/", "./Data/Train3/", "./Data/Train4/", "./Data/Train5/"]
folder = "./Data/NTUH_0722/"
path_list = [folder+"Train1/", folder+"Train2/", folder+"Train3/", folder+"Train4/"]
fd = open("./Fingerprint.txt", 'w')
#total_count = 0

for path in path_list:
    count = 0
    for file in sorted(os.listdir(path)):
        pos = file.strip('.txt')
        print(f"Position,{pos}", file = fd)
        count += 1
        with open(path+file,'r') as f:
            lines = f.readlines()[1:]
            d = dict()
            
            for line in lines:
                line = line.split('|')
                BSSID = line[2][:-3]
                #print(BSSID)
                if BSSID not in BSSID_list:
                    continue
                RSSI = float(line[3].strip("dBm"))
                if BSSID not in d:
                    d[BSSID] = [RSSI]
                else:
                    d[BSSID].append(RSSI)
                    
            for BSSID in d.keys():
                avg_RSSI = int(sum(d[BSSID]) / len(d[BSSID]))
                print(f"{BSSID},{avg_RSSI}",file = fd)
    print(f"the folder contains position count = {count}")
#print(f"total positions = {total_count}")
fd.close()    

the folder contains position count = 27
the folder contains position count = 27
the folder contains position count = 27
the folder contains position count = 27


In [11]:
def save_fingerprint(dir_path, training_wifi_pos, testing_wifi_pos):
    print('='*50)
    print(dir_path)
    print('training wifi pos shape', training_wifi_pos.shape)
    print('testing wifi pos shape', testing_wifi_pos.shape)

    training_wifi_pos_path = os.path.join(dir_path, 'training_wifi_pos.npy') 
    testing_wifi_pos_path = os.path.join(dir_path, 'testing_wifi_pos.npy') 

    np.save(training_wifi_pos_path, training_wifi_pos)
    np.save(testing_wifi_pos_path, testing_wifi_pos)

In [12]:
#for preprocessing Fingerprint.txt to training & testing data
def NTUH_2F():
    fingerprint_file_path = "./Fingerprint.txt"
    
    bssid_index = {}
    training_wifi_pos = []
    testing_wifi_pos = []

    with open(fingerprint_file_path, 'r') as f:
        position_index = []
        index = 0
        lines = f.read().splitlines()
        for i in range(len(lines)):
            elements = lines[i].split(',')
            if elements[0] == 'Position':
                position_index.append(i)
            else:
                bssid, rssi = elements
                if bssid not in bssid_index:
                    bssid_index[bssid] = index
                    index += 1
        
        total_APs = len(bssid_index)
        print(f"total ap = {total_APs}")
        testing_set = set()
        
        for p_idx in position_index:
            fingerprint = np.full(total_APs+2, -100, dtype=np.float64)
            _, x, y = lines[p_idx].split(',')
            #pos = (eval(x)/100, eval(y)/100)
            pos = (eval(x), eval(y))
            fingerprint[-2:] = pos

            p_idx += 1
            elements = lines[p_idx].split(',')
            while len(elements) == 2:
                bssid, rssi = elements
                bssid, rssi = bssid_index[bssid], eval(rssi)
                fingerprint[bssid] = rssi
                p_idx += 1
                if p_idx == len(lines): 
                    break
                elements = lines[p_idx].split(',')

            if pos not in testing_set:
                testing_set.add(pos)
                testing_wifi_pos.append(fingerprint)
            else:
                training_wifi_pos.append(fingerprint)
        #print(len(training_wifi_pos))
        #print((testing_wifi_pos))
        training_wifi_pos = np.array(training_wifi_pos, dtype=np.float64)
        testing_wifi_pos = np.array(testing_wifi_pos, dtype=np.float64)
        save_fingerprint("./Dataset/NTUH_2F", training_wifi_pos, testing_wifi_pos)

In [13]:
NTUH_2F()

total ap = 28
./Dataset/NTUH_2F
training wifi pos shape (81, 30)
testing wifi pos shape (27, 30)


In [14]:
#####
#below is just for testing no need to execute
raise

RuntimeError: No active exception to reraise

In [ ]:

def testing():
    fingerprint_file_path = "./Fingerprint2.txt"
    
    bssid_index = {}
    training_wifi_pos = []
    testing_wifi_pos = []

    with open(fingerprint_file_path, 'r') as f:
        position_index = []
        index = 0
        lines = f.read().splitlines()
        for i in range(len(lines)):
            elements = lines[i].split(',')
            if elements[0] == 'Position':
                position_index.append(i)
            else:
                bssid, rssi = elements
                if bssid not in bssid_index:
                    bssid_index[bssid] = index
                    index += 1
        
        total_APs = len(bssid_index)
        print(f"total ap = {total_APs}")
        testing_set = set()
        
        for p_idx in position_index:
            fingerprint = np.full(total_APs+2, -100, dtype=np.float64)
            _, x, y = lines[p_idx].split(',')
            #pos = (eval(x)/100, eval(y)/100)
            pos = (eval(x), eval(y))
            fingerprint[-2:] = pos

            p_idx += 1
            elements = lines[p_idx].split(',')
            while len(elements) == 2:
                bssid, rssi = elements
                bssid, rssi = bssid_index[bssid], eval(rssi)
                fingerprint[bssid] = rssi
                p_idx += 1
                if p_idx == len(lines): 
                    break
                elements = lines[p_idx].split(',')

            if pos not in testing_set:
                testing_set.add(pos)
                testing_wifi_pos.append(fingerprint)
                
            else:
                training_wifi_pos.append(fingerprint)
            print(testing_wifi_pos)

In [ ]:
path1 = "./Data/Train2/-1,14.txt"
path2 = "./Data/Train4/-1,14.txt"
l1 = []
l2 = []
with open(path1,'r') as f1:
    lines = f1.readlines()[1:]
    for line in lines:
        line = line.split('|')
        SSID = line[1]
        if SSID not in ap_list:
            continue
        BSSID = line[2]
        RSSI = line[3].strip("dBm")
        l1.append(SSID+','+BSSID)

with open(path2,'r') as f2:
    lines = f2.readlines()[1:]
    for line in lines:
        line = line.split('|')
        SSID = line[1]
        if SSID not in ap_list:
            continue
        BSSID = line[2]
        RSSI = line[3].strip("dBm")
        l2.append(SSID+','+BSSID)
#print(l1)
#print(l2)

count = 0
for x in l1:
    if x in l2:
        count += 1
print(len(l1))
print(len(l2))
print(count)
        

215
178
9
